In [95]:
import sympy as sp
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt


In [96]:
def get_energies(solver_result: sc.integrate._ivp.ivp.OdeResult,
                 mass: float,
                 length: float,
                 gravity: float):
    t = sp.symbols('t')  # time
    m, l, g = sp.symbols('m l g')        # masses, lengths, gravitational acceleration
    phi = sp.symbols('phi', cls=sp.Function)  # angles as functions of time
    
    # Define the angle functions and their derivatives
    phi = phi(t)
    phi_dot = sp.diff(phi, t)
    
    # Define the Lagrangian
    U = -m * g * l * (1 - sp.cos(phi))
    T = sp.Rational(1, 2) * m * (l * phi_dot)**2
    # Lambdify the energy expressions
    T_func = sp.lambdify([l, m, phi_dot], T, 'numpy')
    U_func = sp.lambdify([l, m, g, phi],U, 'numpy')

    # Extract the solver's results
    phi_values, phi_dot_values = solver_result.y

    # Calculate energies
    kinetic_energies = T_func(length, mass, phi_dot_values)
    potential_energies = U_func(length, mass, gravity, phi_values)
    return kinetic_energies, potential_energies

In [97]:
# Define the symbols
t = sp.symbols('t')  # time
m, l, g = sp.symbols('m l g')        # masses, lengths, gravitational acceleration
phi = sp.symbols('phi', cls=sp.Function)  # angles as functions of time

# Define the angle functions and their derivatives
phi = phi(t)
phi_dot = sp.diff(phi, t)

# Define the Lagrangian
U = m * g * l * (1 - sp.cos(phi))
T = sp.Rational(1, 2) * m * (l * phi_dot)**2
L = T-U


In [98]:
# Apply the Euler-Lagrange equation
EL = sp.diff(sp.diff(L, phi_dot), t) - sp.diff(L, phi)

# Optionally, simplify the equations
EL_simplified = sp.Eq(sp.simplify(EL),0)



In [99]:
EL_simplified

Eq(l*m*(g*sin(phi(t)) + l*Derivative(phi(t), (t, 2))), 0)

In [100]:
# Defining the second order time deriv variables
phi_ddot = sp.diff(phi, t, t)
# Solving the system of EL equations for the second order time derivatives
# N.B. Solutions is a dict: variable -> equation
solutions = sp.solve([EL_simplified], [phi_ddot]) 


In [101]:
# Turning the second order equations into vectorized numpy functions that can efficiently evaluate numerical values
phi1_ddot_lambdified = sp.lambdify([l, g, phi], solutions[phi_ddot], 'numpy')

# Wrapping in outer function to make more recognizable 
def d2_phi_dt2(phi:float, 
               length: float,
               g: float):
    return phi1_ddot_lambdified(length, g, phi)


In [103]:
# Define the ODE function
def single_pendulum(t, y, l, g):
    phi, d_phi_dt = y
    d2_phi_dt2_value = d2_phi_dt2(phi=phi,length=l, g=g)

    return [d_phi_dt, d2_phi_dt2_value]

# Initial conditions: [phi_init, d_phi_dt_init]
phi_init = np.pi/3
initial_conditions = [phi_init, 0]  

# Parameters
l, g, m = 1, 1, 1
params = (l, g)  

# Time span for the simulation
n_times = 1000
t_min, t_max = 0, 5
time_span = np.linspace(start=t_min, stop=t_max, num=n_times)

# Solve the ODEs
solution = sc.integrate.solve_ivp(fun=single_pendulum,
                        t_span=(t_min,t_max),
                        y0=initial_conditions,
                        method='Radau', # Seems to yield lowest nymerical error
                        args=params,
                        t_eval=time_span)


def get_energies(solver_result: sc.integrate._ivp.ivp.OdeResult,
                 mass: float,
                 length: float,
                 gravity: float):
    t = sp.symbols('t')  # time
    m, l, g = sp.symbols('m l g')        # masses, lengths, gravitational acceleration
    phi = sp.symbols('phi', cls=sp.Function)  # angles as functions of time
    
    # Define the angle functions and their derivatives
    phi = phi(t)
    phi_dot = sp.diff(phi, t)
    
    # Define the Lagrangian
    U = m * g * l * (1 - sp.cos(phi))
    T = sp.Rational(1, 2) * m * (l * phi_dot)**2
    # Lambdify the energy expressions
    T_func = sp.lambdify([l, m, phi_dot], T, 'numpy')
    U_func = sp.lambdify([l, m, g, phi],U, 'numpy')

    # Extract the solver's results
    phi_values, phi_dot_values = solver_result.y

    # Calculate energies
    kinetic_energies = T_func(length, mass, phi_dot_values)
    potential_energies = U_func(length, mass, gravity, phi_values)
    return kinetic_energies, potential_energies

kinetic_energies, potential_energies = get_energies(solver_result = solution,
                                                    mass = m,
                                                    length = l, 
                                                    gravity = g)

In [104]:
(kinetic_energies+potential_energies)[0], (kinetic_energies+potential_energies)[-1]

(0.4999999999999999, 0.49997248455397164)